# 🔍 00.03: 기본 RAG 맛보기

## 💡 검색 + 생성이 어떻게 작동하는지 체험!

In [ ]:
# 1. 이전 단계에서 만든 벡터 저장소 재사용
from langchain.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS
from langchain.schema import Document

# 간단한 문서들
documents = [
    "파이썬은 1991년 귀도가 만든 프로그래밍 언어입니다.",
    "머신러닝은 데이터로 패턴을 학습하는 기술입니다.",
    "RAG는 검색과 생성을 합친 AI입니다."
]
docs = [Document(page_content=text) for text in documents]
embeddings = HuggingFaceEmbeddings(model_name="BAAI/bge-m3")
vectorstore = FAISS.from_documents(docs, embeddings)
print("🗃️ 벡터 저장소 준비 완료!")

In [ ]:
# 2. 질문으로 관련 문서 검색하기
question = "RAG가 뭐야?"
results = vectorstore.similarity_search(question, k=1)
context = results[0].page_content
print(f"🔍 '{question}' 검색 결과:")
print(f"→ 찾은 문서: {context}")

In [ ]:
# 3. 검색된 문서를 바탕으로 답변 생성
from transformers import pipeline

# 간단한 생성 모델 (실제론 더 큰 모델 사용)
generator = pipeline('text-generation', 
                    model='gpt2', 
                    max_length=100, 
                    do_sample=True, 
                    temperature=0.7)

prompt = f"질문: {question}\n컨텍스트: {context}\n답변:"
response = generator(prompt, max_length=len(prompt)+50)[0]['generated_text']
answer = response[len(prompt):].strip()

print(f"🤖 RAG 답변:")
print(f"→ {answer}")

In [ ]:
# 4. RAG 과정 정리
print("🎉 기본 RAG 체험 완료!")
print("✅ 1단계: 질문 입력")
print("✅ 2단계: 관련 문서 검색")
print("✅ 3단계: 컨텍스트로 답변 생성")
print("💡 다음: 00.04에서 고급 RAG 기능들 맛보기!")